In [394]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.transform import resize
from glob import glob
from tqdm import tqdm
from torchmetrics.classification import BinaryAUROC
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

In [395]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [396]:
def RetinaTransform(image):
    # Bringing image to 0-1 range, #ATTENTION: CHECK IF ALL IMAGES min max is 0 and 255
    image = image / 255
    
    
    image = resize(image, (64,64),anti_aliasing=False) # resizing image because original does not fit in memory.
    
    
    # Bringing data to CHW format
    # N is a batch size, C denotes a number of channels, 
    # H is a height of input planes in pixels, and W is width in pixels.
    image = image.transpose([2,0,1])
    
    #Fixing dtype to avoid runtime error and save memory
    image = torch.tensor(image ,dtype=torch.float32)
    
    return image

In [397]:
# # Load dataset into memory
# cuda_image_array = []
# data_folder = "Evaluation_Set/Validation"
# label_path = "Evaluation_Set/RFMiD_Validation_Labels.csv"
# label_frame = pd.read_csv(label_path)
# local_transform = RetinaTransform

# for image_name in tqdm(glob(data_folder+"/*")):
#     image = io.imread(image_name)

#     label_frame_index = int(image_name.split("/")[-1].split(".")[0])
#     label = label_frame[label_frame["ID"]==label_frame_index]["Disease_Risk"].values[0]

#     image = local_transform(image)
#     label =  torch.tensor(label ,dtype=torch.float32)

#     cuda_image_array.append((image.to(device),label.to(device)))

In [398]:
class CicikNet(nn.Module):
    def __init__(self):
        super(CicikNet, self).__init__()

        
        self.lrelu = nn.LeakyReLU(0.1)
        # First 2D convolutional layer, taking in 1 input channel (image),
        # outputting 32 convolutional features, with a square kernel size of 3
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1,padding=1)
        # Second 2D convolutional layer, taking in the 32 input layers,
        # outputting 64 convolutional features, with a square kernel size of 3
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=5, stride=1,padding=2)


        # First fully connected layer
        self.fc1 = nn.Linear(64*64, 128)
        # Second fully connected layer that outputs our 10 labels
        self.fc2 = nn.Linear(128, 1)
        
    def forward(self, x):
        x = torch.flatten(self.conv1(x),1)
        #x2 = torch.flatten(self.conv2(x),1)
        
#         x = torch.cat([x1,x2],dim=1)

        x = self.fc1(x)
        x = self.lrelu(x)

        
        x = self.fc2(x)
        
        output = torch.sigmoid(x)
        return output

net = CicikNet().to(device)
# print(net)

In [399]:
class RetinaDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data_indices, transform=None):
        
        self.data = [cuda_image_array[idx] for idx in data_indices]
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [400]:
kf = KFold(n_splits=2)

for idx, (train_index, test_index) in enumerate(kf.split(cuda_image_array)):
    train_dataset = RetinaDataset(data_indices=train_index)
    test_dataset = RetinaDataset(data_indices=test_index)
    
    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=0)
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)
    break

In [401]:
EPOCHS = 100
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
bauc = BinaryAUROC(thresholds=None)

In [402]:
# def plot_grad_flow(named_parameters):
#     ave_grads = []
#     layers = []
#     for n, p in named_parameters:
#         if(p.requires_grad) and ("bias" not in n):
#             layers.append(n)
#             ave_grads.append(p.grad.abs().mean())
#     plt.plot(ave_grads, alpha=0.3, color="b")
#     plt.hlines(0, 0, len(ave_grads)+1, linewidth=1, color="k" )
#     plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
#     plt.xlim(xmin=0, xmax=len(ave_grads))
#     plt.xlabel("Layers")
#     plt.ylabel("average gradient")
#     plt.title("Gradient flow")
#     plt.grid(True)

In [403]:
def model_train_loop(net,train_dataloader):
    
    running_loss = 0.0
    predictions = []
    ground_truth = []
    
    for idx, data in enumerate(train_dataloader, 0):
        # get the inputs
        # print(i,data)
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs.flatten(), labels)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=10)
        
        optimizer.step()
        running_loss += loss.item()
        
        
        
        predictions.append(outputs.flatten())
        ground_truth.append(labels.flatten())
        
    # net.to('cpu')
    # plot_grad_flow(net.named_parameters())
    # plt.show()
    # net.to(device)
    
    predictions=torch.cat(predictions)
    ground_truth = torch.cat(ground_truth)
    
    
    healthy_f1 = f1_score((predictions>0.5).float().cpu(),ground_truth.cpu(),pos_label=0)
    rocauc = bauc(predictions,ground_truth)
    print('[%d, %5d] loss: %.3f, ROC: %.3f, F1_healthy: %.3f\n' % (epoch + 1, idx + 1, running_loss, rocauc, healthy_f1))

In [404]:
def model_test_loop(net, test_dataloader):
    
    with torch.no_grad():
        
        running_loss = 0.0
        test_predictions = []
        test_ground_truth = []
        
        for idx, data in enumerate(test_dataloader, 0):

            inputs, labels = data

            outputs = net(inputs)
            loss = criterion(outputs.flatten(), labels)
            running_loss += loss.item()

            test_predictions.append(outputs.flatten())
            test_ground_truth.append(labels.flatten())

        
        
        test_predictions=torch.cat(test_predictions)
        test_ground_truth = torch.cat(test_ground_truth)


        healthy_f1 = f1_score((test_predictions>0.5).float().cpu(),test_ground_truth.cpu(),pos_label=0)
        rocauc = bauc(test_predictions,test_ground_truth)
        print('[%d, %5d] Validation loss: %.3f, ROC: %.3f, F1_healthy: %.3f \n' % (epoch + 1, idx + 1, running_loss, rocauc, healthy_f1))


In [405]:
# f1_score((predictions>0.5).float().cpu(),ground_truth.cpu(),pos_label=0)

In [406]:
# print(len(train_dataloader))

In [407]:
# MODEL learns the trivial case of all 1s, which means it has capacity to learn:

for epoch in range(EPOCHS):  # loop over the dataset multiple times

    model_train_loop(net,train_dataloader)
    model_test_loop(net,test_dataloader)

    
# print('Finished Training')

[1,   320] loss: 190.434, ROC: 0.533, F1_healthy: 0.094

[1,   320] Validation loss: 145.027, ROC: 0.667, F1_healthy: 0.000 

[2,   320] loss: 181.374, ROC: 0.542, F1_healthy: 0.000

[2,   320] Validation loss: 149.290, ROC: 0.602, F1_healthy: 0.000 

[3,   320] loss: 173.241, ROC: 0.596, F1_healthy: 0.000

[3,   320] Validation loss: 142.005, ROC: 0.686, F1_healthy: 0.000 

[4,   320] loss: 170.777, ROC: 0.622, F1_healthy: 0.026

[4,   320] Validation loss: 142.386, ROC: 0.645, F1_healthy: 0.000 

[5,   320] loss: 166.549, ROC: 0.648, F1_healthy: 0.186

[5,   320] Validation loss: 140.292, ROC: 0.663, F1_healthy: 0.174 

[6,   320] loss: 162.977, ROC: 0.662, F1_healthy: 0.280

[6,   320] Validation loss: 139.181, ROC: 0.668, F1_healthy: 0.350 

[7,   320] loss: 161.006, ROC: 0.672, F1_healthy: 0.340

[7,   320] Validation loss: 137.517, ROC: 0.682, F1_healthy: 0.361 

[8,   320] loss: 158.801, ROC: 0.686, F1_healthy: 0.350

[8,   320] Validation loss: 135.954, ROC: 0.693, F1_healthy: 

KeyboardInterrupt: 